In [1]:
import pandas as pd
import matplotlib.pyplot as plt 

import keras
import numpy as np

from keras.datasets import mnist

from keras.models import Sequential
from keras.layers import Dense,Conv2D ,MaxPool2D ,Flatten, Dropout

from keras.models import load_model



In [2]:
(x_train,y_train) , (x_test,y_test) = mnist.load_data()

In [3]:
x_train.shape , y_train.shape , x_test.shape , y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [4]:
#pre process the image
x_train = x_train.astype(np.float32)/255
x_test = x_test.astype(np.float32)/255
x_train = np.expand_dims(x_train,-1)
x_test = np.expand_dims(x_test,-1)

In [5]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)


In [6]:
model = Sequential()

In [7]:
model.add(Conv2D(32,(3,3),input_shape = (28,28,1), activation = 'relu'))
model.add(MaxPool2D((2,2)))

model.add(Conv2D(64,(3,3), activation = 'relu'))
model.add(MaxPool2D((2,2)))

model.add(Flatten())
model.add(Dropout(0.25))

model.add(Dense(10,activation = "softmax"))




c:\Users\Hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        16,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,826 (136.04 KB)

 Trainable params: 34,826 (136.04 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
model.compile(optimizer = 'adam',loss = keras.losses.categorical_crossentropy,metrics= ['accuracy'])

In [10]:
# callbacks

from keras.callbacks import EarlyStopping ,ModelCheckpoint

es = EarlyStopping(monitor = 'val_acc',mode = 'max' ,min_delta = 0.01, patience = 4 , verbose =1  )

mc = ModelCheckpoint("./bestmodel.keras",monitor = "val_acc", verbose = 1 , save_best_only = True,mode='max')

cb = [es,mc]

In [11]:
his = model.fit(x_train,y_train,epochs =5 , validation_split =0.3 , callbacks = cb)

Epoch 1/5
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 21s 14ms/step - accuracy: 0.8576 - loss: 0.4583 - val_accuracy: 0.9718 - val_loss: 0.0897
Epoch 2/5
   5/1313 ━━━━━━━━━━━━━━━━━━━━ 17s 14ms/step - accuracy: 0.9910 - loss: 0.0404 

c:\Users\Hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\callbacks\early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: accuracy,loss,val_accuracy,val_loss
  current = self.get_monitor_value(logs)
c:\Users\Hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\callbacks\model_checkpoint.py:206: UserWarning: Can save best model only with val_acc available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


1313/1313 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - accuracy: 0.9776 - loss: 0.0726 - val_accuracy: 0.9833 - val_loss: 0.0552
Epoch 3/5
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 19s 14ms/step - accuracy: 0.9815 - loss: 0.0581 - val_accuracy: 0.9829 - val_loss: 0.0556
Epoch 4/5
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - accuracy: 0.9854 - loss: 0.0486 - val_accuracy: 0.9859 - val_loss: 0.0492
Epoch 5/5
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 23s 18ms/step - accuracy: 0.9875 - loss: 0.0395 - val_accuracy: 0.9868 - val_loss: 0.0436


In [12]:
model.save("my_model.keras")
model_S = keras.models.load_model(r"D:\ipython\project\my_model.keras")


In [13]:
score = model_S.evaluate(x_test,y_test)

print(score[1])

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9869 - loss: 0.0395
0.9894999861717224
